In [1]:
"""This script shows how to extract frames having the same temperature
from replica exchange MD run. You can do it with cpptraj but this shows how easily
to write new script with pytraj
# TODO : check typos for DOCo
"""

import unittest
from array import array
from glob import glob
from pytraj.base import *
from pytraj import io as mdio
from pytraj.utils.check_and_assert import assert_almost_equal

def get_frames_same_T():
   # create a list of all remd trajs
   flist = glob("../tests/data/Test_RemdTraj/rem.nc.*") 

   # make a list of TrajReadOnly instances
   trajlist = []
   for fh in flist:
       topfile = "../tests/data/Test_RemdTraj/ala2.99sb.mbondi2.parm7"

       # load trajectory and append to trajlist
       trajlist.append(mdio.load(fh, topfile))

   # make FrameArray instance that holds 492.2 T frames
   # we need to reserve n_frames to hold the data
   f4922 = FrameArray(n_frames=trajlist[0].n_frames)

   assert f4922.n_frames == trajlist[0].n_frames
   f4922.top = trajlist[0].top.copy()

   # extract frames having T = 492.2
   # use iteration for nested loops
   for traj in trajlist:
       for idx, frame in enumerate(traj):
           if frame.temperature == 492.2:
               # we don't use `append` method since we want to make sure 
               # frames are in the order of simulation time
               f4922[idx] = frame

   # make sure f4922 only hold frames having T = 492.2 K
   arr0 = array('d', [492.2, 492.2, 492.2, 492.2, 492.2, 
                     492.2, 492.2, 492.2, 492.2, 492.2])
   assert f4922.temperatures == arr0

   # make sure we reproduce cpptraj output
   cpptraj = mdio.load("../tests/data/Test_RemdTraj/temp0.crd.492.20", topfile)
   for idx, framepy in enumerate(f4922):
       assert_almost_equal(framepy.coords, cpptraj[idx].coords)
       print("rmsd between pytraj Frame and cpptraj Frame = %s " % framepy.rmsd(cpptraj[idx]))

   print(f4922[5].coords[:10])
   print(cpptraj[5].coords[:10])

   print("YES, we can reproduce cpptraj output")


        

In [2]:
get_frames_same_T()

rmsd between pytraj Frame and cpptraj Frame = 0.0004537857325777221 
rmsd between pytraj Frame and cpptraj Frame = 0.00040011085473567147 
rmsd between pytraj Frame and cpptraj Frame = 0.0004965490165713683 
rmsd between pytraj Frame and cpptraj Frame = 0.0004901392777290847 
rmsd between pytraj Frame and cpptraj Frame = 0.00048039576004936676 
rmsd between pytraj Frame and cpptraj Frame = 0.0004474136708874317 
rmsd between pytraj Frame and cpptraj Frame = 0.0004878846620305841 
rmsd between pytraj Frame and cpptraj Frame = 0.0005227524683447184 
rmsd between pytraj Frame and cpptraj Frame = 0.00046504114463576655 
rmsd between pytraj Frame and cpptraj Frame = 0.0004950090575904039 
array('d', [8.037311553955078, 5.207451820373535, 1.3513193130493164, 7.069351673126221, 5.673007011413574, 1.1658153533935547, 7.176811695098877, 6.072388648986816, 0.15732811391353607, 6.865853786468506])
array('d', [8.037, 5.207, 1.351, 7.069, 5.673, 1.166, 7.177, 6.072, 0.157, 6.866])
YES, we can repro